In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!pip install keras
!pip install np_utils

    100% |████████████████████████████████| 61kB 2.4MB/s 
    100% |████████████████████████████████| 829kB 1.4MB/s 
  Running setup.py bdist_wheel for np-utils ... - \ done
  Stored in directory: /content/.cache/pip/wheels/55/89/c2/e7b9721b946bbfdac6d1f43fa872b823020dbb58754fd75e1c
  Running setup.py bdist_wheel for future ... - \ | / done
  Stored in directory: /content/.cache/pip/wheels/c2/50/7c/0d83b4baac4f63ff7a765bd16390d2ab43c93587fac9d6017a
Successfully built np-utils future
  Found existing installation: future 0.15.2
    Uninstalling future-0.15.2:
      Successfully uninstalled future-0.15.2


In [0]:
import os
os.chdir('drive/app')

In [13]:

# coding: utf-8

# In[1]:

from __future__ import print_function
import keras
import time
from PIL import Image
import numpy as np
import os


# In[2]:

from keras import backend
from keras.models import Model
from keras.applications.vgg16 import VGG16


# In[3]:

from scipy.optimize import fmin_l_bfgs_b
from scipy.misc import imsave, imread, imresize


# In[4]:

# In[5]:

# In[6]:

# In[7]:

# In[8]:

# In[9]:

# In[10]:

# In[11]:

# In[12]:

# In[13]:

def content_loss(content, new_content):
    return backend.sum(backend.square(new_content - content))

# In[14]:

# In[15]:

def gram_matrix(x):
    features = backend.batch_flatten(backend.permute_dimensions(x, (2, 0, 1)))
    gram = backend.dot(features, backend.transpose(features))
    return gram

def style_loss(style, new_style):
    S = gram_matrix(style)
    NS = gram_matrix(new_style)

    channels = 3
    size = height * width

    return backend.sum(backend.square(S - NS)) / (4. * (channels ** 2) * (size ** 2))


# In[16]:

# In[17]:

def total_variation_loss(x):
    a = backend.square(x[:, :height-1, :width-1, :] - x[:, 1:, :width-1, :])
    b = backend.square(x[:, :height-1, :width-1, :] - x[:, :height-1, 1:, :])
    return backend.sum(backend.pow(a + b, 1.25))

# In[18]:

# In[19]:

#grads = backend.gradients(loss, new_content_image)
#grads = backend.gradients(loss, new_style_image)
#grads = backend.gradients(loss, new_combination_image)


# In[20]:

#outputs = [loss]
#outputs += grads

#f_outputs = backend.function([new_content_image], outputs)

#f_outputs = backend.function([new_style_image], outputs)

#f_outputs = backend.function([new_combination_image], outputs)


# In[21]:

def eval_loss_and_grads(x):
    x = x.reshape((1, height, width, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    grad_values = outs[1].flatten().astype('float64')
    return loss_value, grad_values


# In[22]:

class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grad_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

#evaluator = Evaluator()

def load_mask(mask_image_path, shape):
    mask = imread(mask_image_path, mode='L')
    width, height, _ = shape
    mask = imresize(mask, (width, height), interp='bicubic').astype('float32')

    mask[mask <= 127] = 0
    mask[mask > 128] = 255

    maximum = np.amax(mask)
    mask /= maximum

    return mask

def merge(background_image, generated_image, mask_image):
    width, height, channels = generated_image.shape

    for i in range(width):
        for j in range(height):
            if mask_image[i, j] != 0.:
                background_image[i, j, :] = generated_image[i, j, :]

    return background_image

# In[23]:

#-------------------------------------------------------------------------------
if __name__ == '__main__':

    height = 512
    width = 512

    content_image_path = input('Enter the path of content image along with its extension:\n')
    content_image = Image.open(content_image_path).resize((height, width))

    no_style_images = int(input('Enter the no. of style images you wish to use on your content image:\n'))
    style_image = []
    for i in range(no_style_images):
        style_image_path = input('Enter the path of style image ' + str(i+1) + ' along with its extension:\n')
        style_image.append(Image.open(style_image_path).resize((height, width)))

    content_array = np.asarray(content_image, dtype='float32')
    content_array = np.expand_dims(content_array, axis=0)
    #print(content_array)
    print(content_array.shape)

    style_array = []

    for i in range(no_style_images):
        style_array.append(np.expand_dims(np.asarray(style_image[i], dtype='float32'), axis=0))
        #print(style_array)
        print(style_array[i].shape)

    content_array[:, :, :, 0] -= 103.939
    content_array[:, :, :, 1] -= 116.779
    content_array[:, :, :, 2] -= 123.68
    content_array = content_array[:, :, :, ::-1]

    for i in range(no_style_images):
        style_array[i][:, :, :, 0] -= 103.939
        style_array[i][:, :, :, 1] -= 116.779
        style_array[i][:, :, :, 2] -= 123.68
        style_array[i] = style_array[i][:, :, :, ::-1]

    content_image = backend.variable(content_array)

    for i in range(no_style_images):
        style_image[i] = backend.variable(style_array[i])

    new_combination_image = backend.placeholder((1, height, width, 3))

    input_tensor_list = [content_image]
    for i in range(no_style_images):
        input_tensor_list.append(style_image[i])
    input_tensor_list.append(new_combination_image)
    input_tensor = backend.concatenate(input_tensor_list, axis=0)

    model = VGG16(input_tensor=input_tensor, weights='imagenet', include_top=False)

    layers = dict([(layer.name, layer.output) for layer in model.layers])
    #print(layers)

    content_image_weight = 0.02
    style_image_weight = []

    for i in range(no_style_images):
        style_image_weight.append(25.0)

    total_variation_weight = 1.0
    loss = backend.variable(0.)

    layer_features = layers['block2_conv1']
    content_image_features = layer_features[0, :, :, :]
    new_combination_image_features = layer_features[no_style_images+1, :, :, :]
    loss += content_loss(content=content_image_features, new_content=new_combination_image_features)

    feature_layers = ['block1_conv2', 'block2_conv2', 'block3_conv3', 'block4_conv3', 'block5_conv3']

    for layer_name in feature_layers:
        layer_features = layers[layer_name]
        style_features = []
        for i in range(no_style_images):
            style_features.append(layer_features[i+1, :, :, :])

        new_combination_image_features = layer_features[no_style_images+1, :, :, :]
        sl = []
        for i in range(no_style_images):
            sl.append(style_loss(style_features[i], new_combination_image_features))
        for i in range(no_style_images):
            loss += (style_image_weight[i] / len(feature_layers)) * sl[i]

    loss += total_variation_weight * total_variation_loss(new_combination_image)

    grads = backend.gradients(loss, new_combination_image)

    outputs = [loss]
    outputs += grads
    f_outputs = backend.function([new_combination_image], outputs)

    evaluator = Evaluator()

    x = np.random.uniform(0, 255, (1, height, width, 3)) - 128
    iterations = 10

    for i in range(iterations):
        print('Start of iterations of ', i)

        start_time = time.time()
        x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(), fprime=evaluator.grads, maxfun=20)

        print('Current loss value: ', min_val)

        end_time = time.time()
        print('Iteration %d completed in %ds' % (i, end_time - start_time))

        iter_x = np.copy(x)
        iter_x = iter_x.reshape((height, width, 3))
        iter_x = iter_x[:, :, ::-1]
        iter_x[:, :, 0] += 103.939
        iter_x[:, :, 1] += 116.779
        iter_x[:, :, 2] += 123.68
        iter_x = np.clip(iter_x, 0, 255).astype('uint8')

        Image.fromarray(iter_x)

    imsave('starryf3.jpg', iter_x)
    """

    generated_image_path = input('Enter the path of generated image along with its extension:\n')
    generated_image = imread(generated_image_path, mode='RGB')
    generated_image = imresize(generated_image, (height, width))

    background_image_path = input('Enter the path of background image along with its extension:\n')
    background_image = imread(background_image_path, mode='RGB')
    background_image = imresize(background_image, (height, width), interp='bicubic')

    mask_image_path = input('Enter the path of mask image along with its extension:\n')
    mask_image = load_mask(mask_image_path, generated_image.shape)

    final_mask_image = merge(background_image, generated_image, mask_image)
    imsave('final_mask.jpg', final_mask_image)
"""

Enter the path of content image along with its extension:
Dawn Sky.jpg
Enter the no. of style images you wish to use on your content image:
1
Enter the path of style image 1 along with its extension:
The_scream.jpg
(1, 512, 512, 3)
(1, 512, 512, 3)
Start of iterations of  0
Current loss value:  361499800000.0
Iteration 0 completed in 19s
Start of iterations of  1
Current loss value:  171170970000.0
Iteration 1 completed in 18s
Start of iterations of  2
Current loss value:  133307510000.0
Iteration 2 completed in 18s
Start of iterations of  3
Current loss value:  118138920000.0
Iteration 3 completed in 18s
Start of iterations of  4
Current loss value:  112293085000.0
Iteration 4 completed in 18s
Start of iterations of  5
Current loss value:  109128510000.0
Iteration 5 completed in 18s
Start of iterations of  6
Current loss value:  107406340000.0
Iteration 6 completed in 18s
Start of iterations of  7
Current loss value:  106406660000.0
Iteration 7 completed in 18s
Start of iterations of 

In [14]:


generated_image_path = input('Enter the path of generated image along with its extension:\n')
generated_image = imread(generated_image_path, mode='RGB')
generated_image = imresize(generated_image, (height, width))

background_image_path = input('Enter the path of background image along with its extension:\n')
background_image = imread(background_image_path, mode='RGB')
background_image = imresize(background_image, (height, width), interp='bicubic')

mask_image_path = input('Enter the path of mask image along with its extension:\n')
mask_image = load_mask(mask_image_path, generated_image.shape)

final_mask_image = merge(background_image, generated_image, mask_image)
imsave('final_mask.jpg', final_mask_image)

Enter the path of generated image along with its extension:
starryf.jpg


/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


Enter the path of background image along with its extension:
starryf2.jpg
Enter the path of mask image along with its extension:
Dawn-Sky-Mask.jpg
